# Generate a Set of Modules

This notebook is adapted from https://github.com/JonathanKuelz/timor-python/blob/main/tutorials/create_some_modules.ipynb

In [1]:
from datetime import datetime
from pathlib import Path

import numpy as np #numpy
import pinocchio as pin #physics solver

#important imports
from timor.Bodies import Body, Connector, Gender
from timor.Joints import Joint
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ

2024-12-03 20:51:00,481 Timor INFO Loading custom configurations from /home/mscsim/.config/timor.config
2024-12-03 20:51:01,2 Timor INFO Getting robot modrob-gen2.
2024-12-03 20:51:01,4 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


In [2]:
# Let's define some principle parameters
density = 1200  # kg / m3 estimated overall material density
diameter = 80 / 1000  # the diameter of our links [m]
inner_diameter = 60 / 1000  # assume the links are hollow, with 5cm thick walls
sizes = (150 / 1000, 300 / 1000, 450 / 1000)  # we are going to build links in various lengths (sizes) [m]

date = datetime(2022, 9, 15)

ROT_X = Transformation.from_rotation(rotX(np.pi)[:3, :3])  # A shortcut we are going to use frequently
ROT_Y = Transformation.from_rotation(rotY(np.pi / 2)[:3, :3])


# Components

We define some examples of components here:

* Bodies - rigid body (can have different geometries) 
* Joints - connect rigid bodies within modules 
* Connectors - connect modules together. attached to rigid bodies.

In [3]:
# utility functions for calculating the mass and inertia of a cylinder. 
# We can define such helper methods for other geometries as we wish.

def cylinder_mass(l: float, r: float, r_inner: float = 0) -> float:
    """Calculates the mass of a (hollow) cylinder"""
    mass = density * (l * np.pi * r ** 2) #mass = mass/volume * volume
    hollow_mass = density * l * np.pi * r_inner ** 2 #mass = mass/volume * volume
    return mass - hollow_mass # we have a hollow cylinder, so subtract hole from whole


def cylinder_inertia(l: float, r: float, r_inner: float = 0) -> pin.Inertia:
    """Calculates the inertia of a (hollow) cylinder, assuming a centered coordinate system"""
    mass = cylinder_mass(l, r, r_inner)
    lever = np.asarray([0, 0, 0])
    I = np.zeros((3, 3))
    I[0, 0] = (1 / 12) * mass * (3 * (r_inner ** 2 + r ** 2) + l ** 2)
    I[1, 1] = I[0, 0]
    I[2, 2] = .5 * mass * (r_inner ** 2 + r ** 2)

    return pin.Inertia(mass, lever, I)



# Geometry clasess define a shape - https://timor-python.readthedocs.io/en/latest/autoapi/timor/Geometry/index.html
geometry1 = Cylinder({'r': diameter / 2, 'z': sizes[0]},
                    pose=Transformation.from_translation([0, 0, 0])) # cylinder1 along the z-axis
geometry2 = Cylinder({'r': diameter / 2, 'z': sizes[1]}, 
                    pose=Transformation.from_translation([0, 0, 0])) # cylinder2 along the z-axis


# Create a connector - https://timor-python.readthedocs.io/en/latest/autoapi/timor/Bodies/index.html#timor.Bodies.Connector
geo1_connector1 = Connector( connector_id = "geo1_connector_1", #create a unique id for connector
                    body2connector = Transformation.from_translation([0, 0, sizes[0] / 2]), #the transformation from the parent body's frame to this connector's frame. defaults to g = identity matrix
                    parent = None, #the parent body for this connector. will be assigned later if attached to body later
                    gender=Gender.f, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                    connector_type='default', #defines the type of connector. optional
                    size=[diameter]) #defines the "size" of the connector. completely optional

geo1_connector2 = Connector( connector_id = "geo1_connector_2", #create a unique id for connector
                    body2connector = ROT_X @ Transformation.from_translation([0, 0, sizes[0] / 2]), #the transformation from the parent body's frame to this connector's frame. defaults to g = identity matrix
                    parent = None, #the parent body for this connector. will be assigned later if attached to body later
                    gender=Gender.m, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                    connector_type='default', #defines the type of connector. optional
                    size=[diameter]) #defines the "size" of the connector. completely optional

geo2_connector1 = Connector( connector_id = "geo2_connector_1", #create a unique id for connector
                    body2connector = Transformation.from_translation([0, 0, sizes[1] / 2]), #the transformation from the parent body's frame to this connector's frame. defaults to g = identity matrix
                    parent = None, #the parent body for this connector. will be assigned later if attached to body later
                    gender=Gender.f, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                    connector_type='default', #defines the type of connector. optional
                    size=[diameter]) #defines the "size" of the connector. completely optional

geo2_connector2 = Connector( connector_id = "geo2_connector_2", #create a unique id for connector
                    body2connector = ROT_X @ Transformation.from_translation([0, 0, sizes[1] / 2]), #the transformation from the parent body's frame to this connector's frame. defaults to g = identity matrix
                    parent = None, #the parent body for this connector. will be assigned later if attached to body later
                    gender=Gender.m, #Gender.f, Gender.m, or Gender.h. Defaults to hermaphroditic
                    connector_type='default', #defines the type of connector. optional
                    size=[diameter]) #defines the "size" of the connector. completely optional



# Create a body for each cylinder - https://timor-python.readthedocs.io/en/latest/autoapi/timor/Bodies/index.html
# Each cylinder has two connectors, one at each end of the cylinder
body1 = Body(body_id = "test_body_1", #create a unique id for this component
            collision = geometry1, #the collision hitbox uses a Geometry instance
            visual = geometry1, #specifies the Geometry to use for visualizing. Will default to collision.
            connectors = [geo1_connector1, geo1_connector2], #defines what connectors are on this body
            inertia = cylinder_inertia(sizes[0], diameter / 2, inner_diameter / 2), #define the inertia (pinocchio object)
            in_module = None #define what module this body is in. updated automatically if body is added to module later 
            )

body2 = Body(body_id = "test_body_2", #create a unique id for this component
            collision = geometry2, #the collision hitbox uses a Geometry instance
            visual = geometry2, #specifies the Geometry to use for visualizing. Will default to collision.
            connectors = [geo2_connector1, geo2_connector2], #defines what connectors are on this body
            inertia = cylinder_inertia(sizes[1], diameter / 2, inner_diameter / 2), #define the inertia (pinocchio object)
            in_module = None #define what module this body is in. updated automatically if body is added to module later 
            )

joint = Joint(joint_id = "joint_1", #create a unique id for this component
            parent_body = body1, #every joint links a parent body to a child body
            child_body = body2, #every joint links a parent body to a child body
            in_module = None,
            q_limits = (-np.pi, np.pi), #the numerical range for this joint
            torque_limit= np.inf,
            velocity_limit = np.inf,
            acceleration_limit = np.inf,
            parent2joint = Transformation.neutral(), #transformation between base frame of parent link and this joint 
            joint2child = ROT_Y @ Transformation.from_translation([0, 0, sizes[1] / 2]), #transformation between this joint and child link
            joint_type = "revolute"
            )

print(body1.connectors, body2.connectors)





ConnectorSet({<timor.Bodies.Connector object at 0x7fe6195e81c0>, <timor.Bodies.Connector object at 0x7fe6195ff5b0>}) ConnectorSet({<timor.Bodies.Connector object at 0x7fe6195ff490>, <timor.Bodies.Connector object at 0x7fe6195ff610>})


## Creating a module

Modules are composed of the basic components - bodies, joints, and connectors. 

As an example, we start with L-shaped links. These are 2 hollow cylinders connected by a joint.

When creating a module, make sure:
* It has a unique ID within the module set
* Each body in the module (here, it's only one body per module) has a unique ID
* Each connector in the module has a unique ID
* The connector orientations follow the Timor definition. (Their z-axes are pointing away from the module) 

In [4]:
#create a module

module_header = ModuleHeader(ID='T-link',
                                name='T Link Module',
                                date=date,
                                author=['Calix Tang'],
                                email=['calix@berkeley.edu'],
                                affiliation=['UC Berkeley']
                                )

# https://timor-python.readthedocs.io/en/latest/autoapi/timor/Module/index.html
t_link_module = AtomicModule(module_header, #module header definition
                            bodies = (body1, body2), #add all bodies
                            joints = [joint] #add all joints
                            )

print(t_link_module.available_connectors)

{('T-link', 'test_body_2', 'geo2_connector_1'): <timor.Bodies.Connector object at 0x7fe6195ff490>, ('T-link', 'test_body_2', 'geo2_connector_2'): <timor.Bodies.Connector object at 0x7fe6195ff610>, ('T-link', 'test_body_1', 'geo1_connector_2'): <timor.Bodies.Connector object at 0x7fe6195e81c0>, ('T-link', 'test_body_1', 'geo1_connector_1'): <timor.Bodies.Connector object at 0x7fe6195ff5b0>}


In [5]:
#Create a ModulesDB - a collection of unique modules. For now, we only have 1.
db = ModulesDB([t_link_module])
db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


For the **base** and the **end effector** we are going with an easy simplification: The base will just be a box, the end-effector is abstracted as a small sphere, so we can visually identify it's position in visualizations.

In [6]:
def base() -> ModulesDB:
    """Creates a base connector attached to a box."""
    l = .1
    geometry = Box({'x': l, 'y': l, 'z': l}, pose=Transformation.from_translation([0, 0, l / 2]))
    c_world = Connector('base', ROT_X, gender=Gender.f, connector_type='base', size=[diameter])
    c_robot = Connector('base2robot', gender=Gender.m, connector_type='default', size=[diameter],
                        body2connector=Transformation.from_translation([l / 2, 0, l / 2]) @ rotY(np.pi / 2) @ rotZ(np.pi))
    return ModulesDB({
        AtomicModule(ModuleHeader(ID='base', name='Base', author=['Jonathan Külz'], date=date,
                                  email=['jonathan.kuelz@tum.de'], affiliation=['Technical University of Munich']),
                     [Body('base', collision=geometry, connectors=[c_world, c_robot])])
    })


def eef() -> ModulesDB:
    """Creates a simplified end effector module."""
    geometry = Sphere({'r': diameter / 5}, pose=Transformation.from_translation([0, 0, diameter / 2]))
    c_robot = Connector('robot2eef', ROT_X, gender=Gender.f, connector_type='default', size=[diameter])
    c_world = Connector('end-effector', gender=Gender.m, connector_type='eef',
                        body2connector=Transformation.from_translation([0, 0, diameter / 2]))
    return ModulesDB({
        AtomicModule(ModuleHeader(ID='eef', name='Demo EEF', author=['Jonathan Külz'], date=date,
                                  email=['jonathan.kuelz@tum.de'], affiliation=['Technical University of Munich']),
                     [Body('EEF', collision=geometry, connectors=[c_robot, c_world])])
    })

## Last but not least: Build some Joints

We are going to define two joints:

    1. A linear joint which is abstracted by two cylinders of different radius moving relative to each other along their longitudal axis
    2. A prismatic/revolute joint, shaped like one of the L-shaped links but with a relative movement between the two cylinders

In [7]:
def linear_joint() -> ModulesDB:
    """Creates an I-shaped prismatic joint"""
    part_length = 100 / 1000
    joint_limit = .6 * part_length
    proximal = Body('J1_proximal', collision=Cylinder({'r': diameter / 2, 'z': part_length}),
                    connectors=[Connector('J1_proximal',
                                          ROT_X @ Transformation.from_translation([0, 0, part_length / 2]),
                                          gender=Gender.f,
                                          connector_type='default',
                                          size=[diameter])],
                    inertia=cylinder_inertia(part_length, diameter / 2, inner_diameter / 2)
                    )
    distal = Body('J1_distal', collision=Cylinder({'r': 0.8 * diameter / 2, 'z': part_length + joint_limit}),
                  connectors=[Connector('J1_distal',
                                        Transformation.from_translation([0, 0, (part_length + joint_limit) / 2]),
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
                  inertia=cylinder_inertia(part_length * 2, .8 * diameter / 2, .8 * inner_diameter / 2)
                  )
    joint = Joint(
        joint_id='Prismatic',
        joint_type='prismatic',
        parent_body=proximal,
        child_body=distal,
        q_limits=[0, joint_limit],
        parent2joint=Transformation.from_translation([0, 0, part_length / 2]),  # Here we define where the movement axis of the joint is placed
        joint2child=Transformation.from_translation([0, 0, (part_length - joint_limit) / 2])  # Here we define where the movement axis of the joint is placed
    )

    module_header = ModuleHeader(ID='J1',
                                 name='Prismatic Joint',
                                 date=date,
                                 author=['Jonathan Külz'],
                                 email=['jonathan.kuelz@tum.de'],
                                 affiliation=['Technical University of Munich']
                                 )
    return ModulesDB({
        AtomicModule(module_header, [proximal, distal], [joint])
    })

In [8]:
def revolute_joint() -> ModulesDB:
    """Creates an L-shaped joint"""
    length = 150 / 1000
    proximal_body = ComposedGeometry((Cylinder({'r': diameter / 2, 'z': length}),
                                      Sphere({'r': diameter / 2}, pose=Transformation.from_translation([0, 0, length / 2])))
                                     )
    proximal = Body('J2_proximal', collision=proximal_body,
                    connectors=[Connector('J2_proximal',
                                          ROT_X @ Transformation.from_translation([0, 0, length / 2]),
                                          gender=Gender.f,
                                          connector_type='default',
                                          size=[diameter])],
                    inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)
                    )
    distal = Body('J2_distal', collision=Cylinder({'r': diameter / 2, 'z': 150 / 1000}),
                  connectors=[Connector('J2_distal',
                                        Transformation.from_translation([0, 0, length / 2]),
                                        gender=Gender.m,
                                        connector_type='default',
                                        size=[diameter])],
                  inertia=cylinder_inertia(length, diameter / 2, inner_diameter / 2)
                  )
    joint = Joint(
        joint_id='Revolute',
        joint_type='revolute',
        parent_body=proximal,
        child_body=distal,
        q_limits=np.array([-np.pi, np.pi]),
        torque_limit=1000,
        acceleration_limit=5,
        velocity_limit=10,
        parent2joint=Transformation.from_translation([0, 0, length / 2]) @ Transformation.from_rotation(rotY(np.pi / 2)[:3, :3]),
        joint2child=Transformation.from_translation([0, 0, length / 2])
    )

    module_header = ModuleHeader(ID='J2',
                                 name='Revolute Joint',
                                 date=date,
                                 author=['Jonathan Külz'],
                                 email=['jonathan.kuelz@tum.de'],
                                 affiliation=['Technical University of Munich']
                                 )
    return ModulesDB({
        AtomicModule(module_header, [proximal, distal], [joint])
    })

## It's done: Now let's see the modules

We are combining all modules defined to a common database.

Timor offers some visualization capabilities for debugging created databases. It shows the modules, but also all important reference frames defined - take a look, change some parameters and inspect how the visualized modules behave.

In [10]:
db = revolute_joint()
db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
